In [ ]:
#| hide
import os
import joblib

# Syncing through nbdev

> Showcasing the syncing capability

In [ ]:
%load_ext nbmodular.core.cell2func

In [ ]:
%%function myf
print ('hello')
a = 3

> /home/jaumeamllo/workspace/mine/nbmodular/nbmodular/core/cell2func.py(243)run_code_and_collect_locals()
    241         if not is_test_function:
    242             pdb.set_trace()
--> 243             get_old_variables_code = f'\nfrom nbmodular.core.cell2func import keep_variables\nkeep_variables ("previous_values", locals ())'
    244             get_ipython().run_cell(get_old_variables_code)
    245             if store_values:



ipdb>  c


hello
> /home/jaumeamllo/workspace/mine/nbmodular/nbmodular/core/cell2func.py(287)match_variables_and_locals()
    285         pdb.set_trace()
    286         # previous variables / values
--> 287         self.previous_variables = [k for k in self.previous_variables if k in self.previous_values]
    288         self.previous_variables += [k for k in self.argument_variables if k in self.previous_values and k not in self.previous_variables]
    289         self.previous_variables += [k for k in self.created_variables if k in self.previous_values and k in self.loaded_names+self.argument_variables and k not in self.previous_variables]



ipdb>  c


Stored the following local variables in the myf current_values dictionary: ['a']
> /home/jaumeamllo/workspace/mine/nbmodular/nbmodular/core/cell2func.py(316)match_variables_and_locals()
    314         pdb.set_trace()
    315         keys_update_code=(
--> 316 f'''
    317 import joblib
    318 keys = joblib.load ('function_processor_keys.pk')



ipdb>  c


In [ ]:
%print myf

def myf():
    print ('hello')
    a = 3
    return a



In [ ]:
myf_info.original_cell

"print ('hello')\na = 3\n"

In [ ]:
%%function myf --merge
b = 4

> /home/jaumeamllo/workspace/mine/nbmodular/nbmodular/core/cell2func.py(243)run_code_and_collect_locals()
    241         if not is_test_function:
    242             pdb.set_trace()
--> 243             get_old_variables_code = f'\nfrom nbmodular.core.cell2func import keep_variables\nkeep_variables ("previous_values", locals ())'
    244             get_ipython().run_cell(get_old_variables_code)
    245             if store_values:



ipdb>  c


> /home/jaumeamllo/workspace/mine/nbmodular/nbmodular/core/cell2func.py(287)match_variables_and_locals()
    285         pdb.set_trace()
    286         # previous variables / values
--> 287         self.previous_variables = [k for k in self.previous_variables if k in self.previous_values]
    288         self.previous_variables += [k for k in self.argument_variables if k in self.previous_values and k not in self.previous_variables]
    289         self.previous_variables += [k for k in self.created_variables if k in self.previous_values and k in self.loaded_names+self.argument_variables and k not in self.previous_variables]



ipdb>  c


Stored the following local variables in the myf current_values dictionary: ['b']
> /home/jaumeamllo/workspace/mine/nbmodular/nbmodular/core/cell2func.py(316)match_variables_and_locals()
    314         pdb.set_trace()
    315         keys_update_code=(
--> 316 f'''
    317 import joblib
    318 keys = joblib.load ('function_processor_keys.pk')



...

ipdb>  


> /home/jaumeamllo/workspace/mine/nbmodular/nbmodular/core/cell2func.py(847)function()
    845         if func in self.function_info and merge:
    846             pdb.set_trace()
--> 847             this_function = self.merge_functions (self.function_info[func], this_function, show=show)
    848 
    849         function_name = this_function.name



ipdb>  ll


    821     def function (
    822         self,
    823         func,
    824         cell,
    825         merge=False,
    826         show=False,
    827         register_pipeline=True,
    828         pipeline_name=None,
    829         write=True,
    830         test=False,
    831         data=False,
    832         **kwargs
    833     ) -> None:
    834 
    835         function_list = (self.test_function_list if test and not data else 
    836                          self.test_data_function_list if test and data else
    837                          self.function_list)
    838         func_name = f'test_{func}' if test else func
    839         for f in function_list:
    840             if f.name == func_name:
    841                 function_list.remove (f)
    842                 break
    843 
    844         this_function = self.create_function_register_and_run_code (func, cell, show=show, test=test, data=data, **kwargs)
    845         if func in self.function_info an

ipdb>  q


In [ ]:
%debug

> /tmp/ipykernel_31423/1885411832.py(3)<dictcomp>()
      1 import joblib
      2 keys = joblib.load ('function_processor_keys.pk')
----> 3 myf_info.current_values = {k: myf_info.current_values[k] for k in keys['current_values_keys']}
      4 myf_info.previous_values = {k: myf_info.previous_values[k] for k in keys['previous_values_keys']}
      5 myf_info.update ({k: keys[k] for k in ['argument_variables', 'read_only_variables', 'previous_variables', 'created_variables']})



ipdb>  up


> /tmp/ipykernel_31423/1885411832.py(3)<module>()
      1 import joblib
      2 keys = joblib.load ('function_processor_keys.pk')
----> 3 myf_info.current_values = {k: myf_info.current_values[k] for k in keys['current_values_keys']}
      4 myf_info.previous_values = {k: myf_info.previous_values[k] for k in keys['previous_values_keys']}
      5 myf_info.update ({k: keys[k] for k in ['argument_variables', 'read_only_variables', 'previous_variables', 'created_variables']})



ipdb>  


*** Oldest frame


...

ipdb>  self


*** NameError: name 'self' is not defined


ipdb>  down


*** Newest frame


ipdb>  q


In [ ]:
%% function myf --merge
c = 5
d = a+b+c

{'input': None, 'unknown_input': True, 'output': None, 'unknown_output': True}
